In [1]:
import os
import shutil
import time
os.environ["TOKENIZERS_PARALLELISM"]="TRUE"
import pandas as pd
from transformers import CLIPTokenizerFast, CLIPProcessor, CLIPModel
import torch
import importlib
import numpy as np
from tqdm import trange, tqdm
import matplotlib.pyplot as plt 
import GA_Clip_utils
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import checkdf
import processGen
importlib.reload(checkdf)
importlib.reload(processGen)

<module 'processGen' from 'c:\\Users\\Lyle\\Documents\\Files\\DeCoDE\\Bike-Rendering-Dataset\\processGen.py'>

In [2]:
biked = pd.read_csv("Biked_Reference_Data/clip_sBIKED_processed.csv", index_col=0)
dataset="clip_s"

In [ ]:
display(biked)

,CS textfield,BB textfield,Stack,Head angle,Head tube length textfield,Seat stay junction0,Seat tube length,Seat angle,DT Length,BB diameter,...,bottle DOWNTUBE0 show OHCLASS: False,bottle DOWNTUBE0 show OHCLASS: True,BELTorCHAIN OHCLASS: 0,BELTorCHAIN OHCLASS: 1,RIM_STYLE front OHCLASS: DISC,RIM_STYLE front OHCLASS: SPOKED,RIM_STYLE front OHCLASS: TRISPOKE,RIM_STYLE rear OHCLASS: DISC,RIM_STYLE rear OHCLASS: SPOKED,RIM_STYLE rear OHCLASS: TRISPOKE
1,430.00,67.000000,565.6,73.000000,135.6,45.0,588.7,72.500000,664.020690,40.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,350.00,-14.500000,565.6,71.500000,80.0,45.0,265.0,71.500000,572.491950,40.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,415.00,4.000000,565.6,72.000000,198.8,45.0,562.0,74.000000,647.103654,38.1,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,375.00,50.000000,565.6,70.500000,91.8,40.0,460.0,73.500000,573.904093,38.1,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
5,431.79,24.000000,565.6,72.100000,83.1,41.0,300.0,72.500000,636.179297,40.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4796,410.00,75.000000,552.0,72.500000,145.0,125.0,520.0,74.000000,644.551458,40.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4797,419.00,42.909345,598.9,65.772099,95.0,90.0,380.0,77.772099,704.278730,40.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4798,410.00,-11.500000,492.5,72.000000,250.0,45.0,550.0,74.000000,547.934671,40.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
4799,410.00,-11.500000,500.0,72.000000,280.4,45.0,550.0,74.000532,542.387502,40.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0


In [ ]:
ub = np.quantile(biked.values, 0.99, axis=0)
lb = np.quantile(biked.values, 0.01, axis=0)


In [ ]:
from scipy.stats import qmc
sampler = qmc.Sobol(d=len(biked.columns), scramble=False)
samples = sampler.random_base2(m=24)
samples = samples * (ub - lb) + lb


In [ ]:
# device = "cuda"
# model_id = "openai/clip-vit-base-patch32"
# processor = CLIPProcessor.from_pretrained(model_id)


# model = CLIPModel.from_pretrained(model_id).to(device)
thread_count = GA_Clip_utils.init_mp(reserve_threads=1)

CPU Thread Count: 32; reserving 1


In [ ]:
subsamples = samples[:2000,:]
subsample_df = pd.DataFrame(subsamples, columns=biked.columns)
res = GA_Clip_utils.get_image(subsample_df, dataset, thread_count, num_views=10, timeout=10)

0.094 fraction valid


Process ForkPoolWorker-4:
Traceback (most recent call last):
  File "/home/mileva/mambaforge/envs/clip/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/mileva/mambaforge/envs/clip/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mileva/mambaforge/envs/clip/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home/mileva/mambaforge/envs/clip/lib/python3.10/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/home/mileva/mambaforge/envs/clip/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/mileva/Documents/Lyle/CLIP Dataset/GA_Clip_utils.py", line 106, in timeout_handler
    raise TimeoutException
GA_Clip_utils.TimeoutException
Process ForkPoolWorker-14:
Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Process ForkPoolWorker-6:
Process Fork

Exception in thread "main" java.util.NoSuchElementException: No line found
	at java.base/java.util.Scanner.nextLine(Scanner.java:1651)
	at basic.bikeCADPro$ExtraListener.actionPerformed(bikeCADPro.java:5034)
	at basic.bikeCADPro.splashScreenDestruct(bikeCADPro.java:684)
	at basic.bikeCADPro.main(bikeCADPro.java:466)


[(tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],
  
          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],
  
          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]]]),
  (255, 255, 255)),
 (tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
  

In [ ]:
# def create_batch(i, batch_size=2**10):
#     start_time = time.time()
#     v
#     reference_valid_idxs, reference_meanvals = GA_Clip_utils.get_mean_embedding(subsample_df, dataset, processor, model, thread_count, num_views=5)
#     reference_meanvals = reference_meanvals.cpu().numpy()
#     meanvals_df = pd.DataFrame(reference_meanvals, index=reference_valid_idxs)
#     if not os.path.exists(f"data/batch_{i}"):
#         os.mkdir(f"data/batch_{i}")
#         os.mkdir(f"data/batch_{i}/Images_png")
#         # os.mkdir(f"data/batch_{i}/Images_svg")
#         os.mkdir(f"data/batch_{i}/BCAD_files")
#     meanvals_df.to_csv(f"data/batch_{i}/embeddings.csv")
#     subsample_df.to_csv(f"data/batch_{i}/parametric.csv")
#     for file in os.listdir("BCAD_gen/"):
#         if file.endswith(".png"):
#             shutil.move(f"BCAD_gen/{file}", f"data/batch_{i}/Images_png/{file}")
#         # elif file.endswith(".svg"):
#         #     shutil.move(f"BCAD_gen/{file}", f"data/batch_{i}/Images_svg/{file}")
#         elif file.endswith(".bcad"):
#             shutil.move(f"BCAD_gen/{file}", f"data/batch_{i}/BCAD_files/{file}")
#     print(f"Batch {i} processed in {time.time() - start_time} seconds")



In [ ]:
# batchsize = 2**13
# for i in range(np.ceil(len(samples)/batchsize).astype(int)):
#     if not os.path.exists(f"data/batch_{i}"):
#         create_batch(i, batchsize)
        

Exception in thread "main" java.util.NoSuchElementException: No line found
	at java.base/java.util.Scanner.nextLine(Scanner.java:1651)
	at basic.bikeCADPro$ExtraListener.actionPerformed(bikeCADPro.java:5034)
	at basic.bikeCADPro.splashScreenDestruct(bikeCADPro.java:684)
	at basic.bikeCADPro.main(bikeCADPro.java:466)
Exception in thread "main" java.util.NoSuchElementException: No line found
	at java.base/java.util.Scanner.nextLine(Scanner.java:1651)
	at basic.bikeCADPro$ExtraListener.actionPerformed(bikeCADPro.java:5034)
	at basic.bikeCADPro.splashScreenDestruct(bikeCADPro.java:684)
	at basic.bikeCADPro.main(bikeCADPro.java:466)
Exception in thread "main" Exception in thread "main" Exception in thread "main" java.util.NoSuchElementException: No line found
java.util.NoSuchElementException: No line found
	at java.base/java.util.Scanner.nextLine(Scanner.java:1651)
	at basic.bikeCADPro$ExtraListener.actionPerformed(bikeCADPro.java:5034)
	at basic.bikeCADPro.splashScreenDestruct(bikeCADPro.j